In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS books(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author TEXT NOT NULL,
        year_published INTEGER,
        genre TEXT);
    ''')
    connection.commit()

In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
    INSERT INTO books (title, author, year_published, genre)
    VALUES('The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction');
    ''')
    cursor.execute('''
    INSERT INTO books (title, author, year_published, genre)
    VALUES('1984', 'George Orwell', 1949, 'Dystopian');
    ''')
    cursor.execute('''
    INSERT INTO books (title, author, year_published, genre)
    VALUES('To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic');
    ''')
    connection.commit()



In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT * FROM books;')
    res = cursor.fetchall()
    for i in res:
        print(i)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT * FROM books WHERE year_published >= 1950;')
    res = cursor.fetchall()
    for i in res:
        print(i)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT * FROM books WHERE title LIKE \'T%\';')
    res = cursor.fetchall()
    for i in res:
        print(i)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT * FROM books ORDER BY year_published;')
    res = cursor.fetchall()
    for i in res:
        print(i)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT * FROM books ORDER BY title LIMIT 2;')
    res = cursor.fetchall()
    for i in res:
        print(i)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT COUNT(*) FROM books;')
    res = cursor.fetchall()
    print(res)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
        SELECT genre, COUNT(*) as book_count
        FROM books
        GROUP BY genre
        ORDER BY book_count DESC;
    ''')
    results = cursor.fetchall()
    for genre, count in results:
        print(f"- {genre}: {count} книг(и)")


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
        UPDATE books
        SET year_published = 1948
        WHERE title = \'1984\'
    ''')
    cursor.execute('SELECT * FROM books WHERE title LIKE \'1984\'')
    results = cursor.fetchall()
    for i in results:
        print(i)

In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
        DELETE FROM books WHERE title LIKE \'The Great Gatsby\'
    ''')
    cursor.execute('SELECT * FROM books')
    results = cursor.fetchall()
    for i in results:
        print(i)


In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
        SELECT name
        FROM sqlite_master
        WHERE type = 'index'
        AND tbl_name = 'books'
        AND sql LIKE '%author%';
    ''')
    indexes = cursor.fetchall()
    if indexes:
        print("Найдены индексы для столбца 'author':")
        for idx in indexes:
            print(f"- {idx[0]}")
    else:
        print("Индексы для столбца 'author' отсутствуют.")

In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    action TEXT NOT NULL,
    book_id INTEGER,
    book_title TEXT,
    changed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
''')
    cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_new_book
AFTER INSERT ON books
BEGIN
    INSERT INTO logs (action, book_id, book_title)
    VALUES ('INSERT', NEW.id, NEW.title);
END;
''')
    cursor.execute('''
INSERT INTO books (title, author, year_published, genre)
VALUES ('Pride and Prejudice', 'Jane Austen', 1813, 'Classic');
''')
    cursor.execute('SELECT * FROM logs;')
    logs = cursor.fetchall()
    for log in logs:
        print(log)
    connection.commit()

In [ ]:
import sqlite3
def transaction():
    try:
        connection = sqlite3.connect('library.db')
        cursor = connection.cursor()
        connection.execute("BEGIN TRANSACTION;")
        cursor.execute('''
        INSERT INTO books (title, author, year_published, genre)
        VALUES ('The Hobbit', 'J.R.R. Tolkien', 1937, 'Fantasy');
        ''')
        cursor.execute('''
        UPDATE books
        SET year_published = 1948
        WHERE title = '1984';
        ''')
        connection.commit()
        print("Транзакция выполнена успешно!")
    except Exception as e:
        connection.rollback()
        print(f"Ошибка: {e}. Транзакция отменена.")

    finally:
        connection.close()

transaction()

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM books WHERE title IN ('The Hobbit', '1984');")
    for row in cursor.fetchall():
        print(row)


In [ ]:
import sqlite3

def age(year):
    from datetime import datetime
    return int(datetime.now().year) - year

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    connection.create_function('age', 1, age)
    cursor.execute('''
    SELECT *, age(year_published) as age FROM books;
    ''')
    res = cursor.fetchall()
    for i in res:
        print(i)

In [ ]:
import sqlite3

def age(year):
    from datetime import datetime
    return int(datetime.now().year) - year

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
    CREATE VIEW IF NOT EXISTS modern_books as SELECT * FROM books WHERE year_published > 1950
    ''')
    cursor.execute('SELECT * FROM modern_books')
    res = cursor.fetchall()
    for i in res:
        print(i)

In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
                SELECT name FROM sqlite_master
                WHERE type='table' AND name='books';
            ''')
    table_exists = cursor.fetchone()
    if table_exists:
        print('Таблица \'books\' существует.')
        cursor.execute('DROP TABLE books;')
        connection.commit()
    else:
        print('Таблица \'books\' не существует.')

    cursor.execute('''
                SELECT name FROM sqlite_master
                WHERE type='table' AND name='books';
            ''')
    table_exists = cursor.fetchone()

    if table_exists:
        print('Таблица \'books\' существует.')
    else:
        print('Таблица \'books\' не существует.')

In [ ]:
import sqlite3

with sqlite3.connect('library.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_books_author_year_published
ON books (author, year_published);
    ''')
    connection.commit()

In [ ]:
import os

path = 'library.db'
if os.path.exists(path):
    os.remove(path)
    print(f"Файл базы данных '{path}' успешно удалён.")
else:
    print(f"Файл '{path}' не существует.")

In [ ]:
import sqlite3
import datetime
def craete_items():
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('''
    CREATE TABLE IF NOT EXISTS items(
    item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    product_name TEXT,
    price DECIMAL(10, 2),
    quantity INTEGER,
    type TEXT
    );
    ''')
        connection.commit()
def create_online():
     with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS online_orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer TEXT,
    order_date DATETIME,
    status TEXT,
    payment_method TEXT);
        ''')
        connection.commit()
def create_phone():
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS phone_orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer TEXT,
    order_date DATETIME,
    status TEXT,
    operator TEXT);
        ''')
        connection.commit()
def create_store():
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS store_orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer TEXT,
    order_date DATETIME,
    status TEXT,
    store_location TEXT);
        ''')
        connection.commit()
def insert_online(customer, status, payment_method):
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        data = (customer, datetime.datetime.now(), status, payment_method)
        cursor.execute('''
        INSERT INTO online_orders (customer, order_date, status, payment_method)
        VALUES(?, ?, ?, ?);
        ''', data)
        connection.commit()
def insert_phone(customer, status, operator):
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        data = (customer, datetime.datetime.now(), status, operator)
        cursor.execute('''
        INSERT INTO phone_orders (customer, order_date, status, operator)
        VALUES(?, ?, ?, ?);
        ''', data)
        connection.commit()
def insert_store(customer, status, store_location):
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        data = (customer, datetime.datetime.now(), status, store_location)
        cursor.execute('''
        INSERT INTO store_orders (customer, order_date, status, store_location)
        VALUES(?, ?, ?, ?);
        ''', data)
        connection.commit()
def insert_items(type, names, prices, qua):
     with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        id = int()
        if type == 'online':
            cursor.execute('SELECT MAX(order_id) FROM online_orders')
            id = int(cursor.fetchall()[0][0])
        elif type == 'phone':
            cursor.execute('SELECT MAX(order_id) FROM phone_orders')
            id = int(cursor.fetchall()[0][0])
        elif type == 'store':
            cursor.execute('SELECT MAX(order_id) FROM store_orders')
            id = int(cursor.fetchall()[0][0])
        else:
            raise ValueError('Неизвестный тип заказа')
        for i in range(len(names)):
            data = (id, names[i], prices[i], qua[i], type)
            cursor.execute('''
             INSERT INTO items (order_id, product_name, price, quantity, type)
        VALUES(?, ?, ?, ?, ?);
             ''', data)
        connection.commit()
def create_customers():
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT,
    phone TEXT,
    name TEXT,
    address TEXT
    );
        ''')
        connection.commit()
def create_order_history():
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS order_history(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER,
    customers_id INTEGER,
    order_type TEXT
    );
        ''')
        connection.commit()
def insert_customers(email, phone, name, adr):
     with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        data = (email, phone, name, adr)
        cursor.execute('''
        INSERT INTO customers (email, phone, name, address)
        VALUES(?, ?, ?, ?);
        ''', data)
        connection.commit()
def insert_order_history(order, customer, type):
     with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        data = (order, customer, type)
        cursor.execute('''
        INSERT INTO order_history (order_id, customers_id, order_type)
        VALUES(?, ?, ?);
         ''', data)
        connection.commit()
def customer_info(id):
    with sqlite3.connect('order.db') as connection:
        cursor = connection.cursor()
        cursor.execute('SELECT * FROM customers WHERE id == ?', (id,))
        r = cursor.fetchall()
        result = dict()
        result['id'] = r[0][0]
        result['name'] = r[0][3]
        result['email'] = r[0][1]
        result['phone'] = r[0][2]
        result['address'] = r[0][4]
        cur_id = r[0][0]
        cursor.execute('SELECT * FROM order_history WHERE customers_id == ?', (cur_id,))
        r = cursor.fetchall()
        order_history = []
        for i in r:
            order_id = i[0]
            order_type = i[-1]
            order = dict()
            t = tuple()
            third = ''
            if order_type == 'online':
                cursor.execute('SELECT * FROM online_orders')
                t = cursor.fetchall()
                third = 'payment_method'
            elif order_type == 'phone':
                cursor.execute('SELECT * FROM phone_orders')
                t = cursor.fetchall()
                third = 'operator'
            elif order_type == 'store':
                cursor.execute('SELECT * FROM phone_orders')
                t = cursor.fetchall()
                third = 'store_location'
            order['id'] = t[0][0]
            order['date'] = t[0][2]
            order['status'] = t[0][3]
            order[third] = t[0][4]
            cursor.execute('SELECT * FROM items WHERE order_id == ? AND type == ?', (order_id, order_type))
            t = cursor.fetchall()
            items = dict()
            for i in t:
                items['name'] = i[2]
                items['price'] = i[3]
                items['quantity'] = i[4]
            order['items'] = items
            order_history.append(order)
        result['order_history'] = order_history
        result['type'] = order_type
        return result
